**Troubleshooting Model Performance**

In this notebook I sought to rerun the models that performed the best in Dec. 2024 and cleaned up some of the useless cells. The goal here was to replicate the results we reported, double check the code worked, and see if I could help troubleshoot new modeling attempts in June '25

Nathan Lanclos
06/19/2025

**Data Processing**

In the first couple of cells, we are just processing the data into train and test sets and scaling the data for training. 

In [42]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Path to data folder
directory = '../GATSol/dataset/'

# Load CSVs
df_train = pd.read_pickle(directory + 'eSol_train.pkl')
df_test = pd.read_pickle(directory + 'eSol_test.pkl')

continuous_column = 'solubility' 
binary_column = 'binary_solubility'

# Identify scalar features
scalar_features = [
    col for col in df_train.columns
    if pd.api.types.is_numeric_dtype(df_train[col]) and col != continuous_column and col != binary_column
]

# Extract scalar features and target for regression and classification
X_train = df_train[scalar_features]
y_train_continuous = df_train[continuous_column]
y_train_binary = df_train[binary_column]
X_test = df_test[scalar_features]
y_test_continuous = df_test[continuous_column]
y_test_binary = df_test[binary_column]

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [43]:
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
print(X_train_scaled_df.shape)

(2019, 67)


In the model below, we are simply fitting a linear regression with the scalar features from sequence and structure feature engines, no embeddings are included. 

In [44]:
# ---- Linear Regression ----
print("\n=== Linear Regression ===")
linear_model = LinearRegression()
linear_model.fit(X_train_scaled, y_train_continuous)

# Get coefficients and sort by importance
linear_coefficients = linear_model.coef_
linear_coef_df = pd.DataFrame({
    'Feature': scalar_features,
    'Coefficient': linear_coefficients
}).sort_values(by='Coefficient', key=abs, ascending=False)

# Print sorted coefficients
print("\nSorted Coefficients (Linear Regression):")
print(linear_coef_df)

# Evaluate the model
linear_r_squared = linear_model.score(X_test_scaled, y_test_continuous)
print(f"\nR^2 Score on Test Set: {linear_r_squared:.4f}")


=== Linear Regression ===

Sorted Coefficients (Linear Regression):
                               Feature  Coefficient
45                           num_atoms     3.754350
46                          total_mass    -2.707929
0                     molecular_weight    -1.541898
61              O_atom_type_proportion     0.725713
59              N_atom_type_proportion    -0.611208
..                                 ...          ...
15        Hydrophobicity_FASG890101-G2    -0.004357
55                       b_factors_min     0.003076
50                 bounding_box_volume     0.001675
26  Normalized van der Waals Volume-G1    -0.000721
48                          num_chains     0.000000

[67 rows x 2 columns]

R^2 Score on Test Set: 0.4291


**Processing and Flattening Embeddings**

In these next few cells, we load in the embeddings and create a new train and test variable to train a decision tree model with some starting parameters 

In [45]:
test_embeddings = df_test[["gene", "embedding", "blosum62_embedding"]]
train_embeddings = df_train[["gene", "embedding", "blosum62_embedding"]]

In [46]:
import numpy as np

# Flatten ESM embeddings
train_embeddings['flattened_embedding'] = train_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)
test_embeddings['flattened_embedding'] = test_embeddings['embedding'].apply(
    lambda x: np.array(x).flatten()
)

# Validate dimensions
print("Train ESM dimensions:", train_embeddings['flattened_embedding'].apply(len).unique())
print("Test ESM dimensions:", test_embeddings['flattened_embedding'].apply(len).unique())


Train ESM dimensions: [1280]
Test ESM dimensions: [1280]


/tmp/ipykernel_87901/577380073.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_embeddings['flattened_embedding'] = train_embeddings['embedding'].apply(
/tmp/ipykernel_87901/577380073.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_embeddings['flattened_embedding'] = test_embeddings['embedding'].apply(


In [47]:
X_train_embeddings = np.array(train_embeddings['flattened_embedding'].tolist())
X_test_embeddings = np.array(test_embeddings['flattened_embedding'].tolist())
X_train_combined = np.hstack([X_train_scaled, X_train_embeddings])
X_test_combined = np.hstack([X_test_scaled, X_test_embeddings])
# Check shapes
print("Train embeddings shape:", X_train_embeddings.shape)  # Should be (n_samples, 1280)
print("Test embeddings shape:", X_test_embeddings.shape)    # Should be (n_samples, 1280)
print("Combined train shape:", X_train_combined.shape)
print("Combined test shape:", X_test_combined.shape)


Train embeddings shape: (2019, 1280)
Test embeddings shape: (660, 1280)
Combined train shape: (2019, 1347)
Combined test shape: (660, 1347)


**XGBoost Modeling**

Below we fit an XGBoost model with the features including embeddings using some manually selected parameters

In [48]:
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# Train the XGBoost regressor
xgb_model = XGBRegressor(
    n_estimators=100,       # Number of trees
    learning_rate=0.1,      # Learning rate
    max_depth=5,            # Maximum depth
    random_state=42         # For reproducibility
)
xgb_model.fit(X_train_combined, y_train_continuous)

# Predict on the training set
y_train_pred = xgb_model.predict(X_train_combined)

# Predict on the test set
y_test_pred = xgb_model.predict(X_test_combined)

# Evaluate the model on the training set
train_mse = mean_squared_error(y_train_continuous, y_train_pred)
train_r2 = r2_score(y_train_continuous, y_train_pred)

# Evaluate the model on the test set
test_mse = mean_squared_error(y_test_continuous, y_test_pred)
test_r2 = r2_score(y_test_continuous, y_test_pred)

# Print the results
print(f"Training Mean Squared Error: {train_mse:.4f}")
print(f"Training R^2 Score: {train_r2:.4f}")
print(f"Test Mean Squared Error: {test_mse:.4f}")
print(f"Test R^2 Score: {test_r2:.4f}")


Training Mean Squared Error: 0.0027
Training R^2 Score: 0.9741
Test Mean Squared Error: 0.0526
Test R^2 Score: 0.4914


**Hyperparameter Tuning**

After the above model was evaluated, we ran a 100 trial Optuna experiment for hyperparameter tuning where we also used 3-fold cross validation. I think this is actually where the confusion likely is based on our conversation at JBEI. I did actually find a mistake in our reporting which is that we reported the best hyperparameters for highest test  (trial 99), so we made a mistake in cherry picking a trial from the Optuna run. 

Regardless, the best trial from Optuna was number 87, which had the settings shown below. Note that the R2 is 0.437, which does not come close to our reported 0.53. This is due to the fact that training with CV will change the model performance. 

***The attached png in the email shows our results when doing the training with Optuna, we actually never pushed test R2 above 0.44 at this point.***

However, our goal was to directly improve on the results from GATsol, which uses a defined test/train split with no CV. Their modeling strategy was to do hyperparameter tuning with CV and then report performance for the model using the single test/train with no CV. They did this because the ~15 or so models preceding this publication all used the same split to enable direct comparison, so we did this as well. 

[I 2024-12-14 16:17:36,056] Trial 87 finished with value: 0.43782824071107224 and parameters: {'n_estimators': 135, 'learning_rate': 0.052121761036791094, 'max_depth': 4, 'subsample': 0.8433759176013462, 'colsample_bytree': 0.869835124409304, 'min_child_weight': 2, 'gamma': 0.1032306334323324}. Best is trial 87 with value: 0.43782824071107224.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# --- Assume X_train_combined, y_train_continuous, X_test_combined, y_test_continuous are loaded ---

# Parameters from Trial 87, which Optuna found to be the best via cross-validation
params_trial_87 = {
    'n_estimators': 135,
    'learning_rate': 0.052121761036791094,
    'max_depth': 4,
    'subsample': 0.8433759176013462,
    'colsample_bytree': 0.869835124409304,
    'min_child_weight': 2,
    'gamma': 0.1032306334323324,
    'random_state': 42  # For reproducibility
}

print("--- Evaluating Best Model Identified by Optuna (Trial 87) ---")

# Initialize and train the model with the best parameters
xgb_model_best = XGBRegressor(**params_trial_87)
xgb_model_best.fit(X_train_combined, y_train_continuous)

# Evaluate the model
y_train_pred_best = xgb_model_best.predict(X_train_combined)
y_test_pred_best = xgb_model_best.predict(X_test_combined)

train_r2_best = r2_score(y_train_continuous, y_train_pred_best)
test_r2_best = r2_score(y_test_continuous, y_test_pred_best)

print(f"Training R^2 Score (Trial 87): {train_r2_best:.4f}")
print(f"Test R^2 Score (Trial 87): {test_r2_best:.4f}\n")

--- Evaluating Best Model Identified by Optuna (Trial 87) ---
Training R^2 Score (Trial 87): 0.8522
Test R^2 Score (Trial 87): 0.5292



**Best Optuna parameters performance FOR TRAIN R2 with no CV**

As shown above, when we take the best parameters from our tuning run and apply this to the single split we get 0.529 (still beats GATsol, but 0.1 under what we reported)

**Best Optuna parameters performance FOR TEST R2 with no CV**

The mistake that we made in our reporting is that we identified the best model by running: best_trial = results_df.loc[results_df['r2_test'].idxmax()]

This line returned the "best" trial with respect to test performance, not train performance. This makes the conclusion we drew incorrect because we just ended up using the parameters that optimized our objective, we should have selected on 'r2_train'.

[I 2024-12-14 16:21:29,824] Trial 99 finished with value: 0.4330881580532266 and parameters: {'n_estimators': 142, 'learning_rate': 0.06235983828438135, 'max_depth': 4, 'subsample': 0.8318125531190607, 'colsample_bytree': 0.8769926991459716, 'min_child_weight': 2, 'gamma': 0.16173707656854563}. Best is trial 87 with value: 0.43782824071107224.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score
from xgboost import XGBRegressor

# --- Assume X_train_combined, y_train_continuous, X_test_combined, y_test_continuous are loaded ---

params_trial_87 = {
    'n_estimators': 142,
    'learning_rate': 0.06235983828438135,
    'max_depth': 4,
    'subsample': 0.8318125531190607,
    'colsample_bytree': 0.8769926991459716,
    'min_child_weight': 2,
    'gamma': 0.16173707656854563,
    'random_state': 42  # For reproducibility
}

print("--- Evaluating Best Model for Test R2 Identified by Optuna (Trial 99) ---")

# Initialize and train the model with the best parameters
xgb_model_best = XGBRegressor(**params_trial_87)
xgb_model_best.fit(X_train_combined, y_train_continuous)

# Evaluate the model
y_train_pred_best = xgb_model_best.predict(X_train_combined)
y_test_pred_best = xgb_model_best.predict(X_test_combined)

train_r2_best = r2_score(y_train_continuous, y_train_pred_best)
test_r2_best = r2_score(y_test_continuous, y_test_pred_best)

print(f"Training R^2 Score (Trial 99): {train_r2_best:.4f}")
print(f"Test R^2 Score (Trial 99): {test_r2_best:.4f}\n")

--- Evaluating Best Model for Test R2 Identified by Optuna (Trial 99) ---
Training R^2 Score (Trial 99): 0.8681
Test R^2 Score (Trial 99): 0.5372



**FINAL THOUGHTS**

When we discussed troubleshooting on 6/18/25 you mentioned that you combined the test/train splits, ran CV, and then evaluated on a new test set. 

If our discussion was in reference to your evaluation on this pipeline then your 0.49 is actually beating our results when doing CV (we were getting 0.437). If you look at the GATsol paper (https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-024-05820-8), they barely pushed 0.40 with CV and only reported their 0.51 on the defined split. If you run the model on that static/defined split with no CV it might be more telling if something else is critically wrong. 

I imagine what is happening is that you actually improved the model by processing Blosum correctly. 

If I am wrong about the CV/splits then I would imagine there is a feature engineering issue that we haven't caught. But I am able to run this notebook without issues and I think I remember you saying you're using the pre-processed .pkl anyway. 